In [ ]:
from nbtemplate import display_header
display_header('PiSoXTolerances.ipynb')

# PiSoX: Error budget and alignment requirements

When the physical hardware for a mission is put together, nothing is perfect. Parts and pieces will always differ slightly in form, shape, and position from the locations assigned to them in the abstract design model. Ray-tracing is one useful method to study how much such misalignments will impact the performance of the instrument and thus to develop a table of alignment requirements. The looser the requirement can be, the cheaper and faster the process is. On the other hand, if the ray-tracing shows that certain elements need to be positioned very precisely, specific alignment procedures and tests might have to be developed. 

In the following, we present results from PiSoX ray-tracing studies. For each alignment parameter, we study six degrees of freedom (three translantions and three roations). In practice, misalignments happen in all six degrees of freedom for all parts of the instrument at the same time. However, computational limitations prohibit us from  exhaustively exploring the full parameter space. Instead, as a first phase, we treat PiSoX as a hirachical collection of many elements (mirror shells, mirror module, CAT gratings, CAT grating assembly, all of which combine into the optics module etc.). We perform simulations for about a dozen elements and for each parameter we typically run simulations for 10-20 values. The full parameter space would thus require $20^{6 * 12} = 5 \times 10^{93}$ simulations. Instead, as a first step, we set up a perfectly aligned instrument and then vary one parameter for one element or one group of elements (e.g. the "x" position of all gratings) at a time. Note that even a perfectly aligend instrument has some limitations that are inharent in the design, such as optical abberations.  We step through diffferent values for each parameter, keeping all other alignments perfect, and run a simulations with 100,000 photons for each step. We inspect the results from simulations and select a value for the accpetable misalignment in each degree of freedom, e.g. the value where the effective area of the channel degrades by no more than 10\%. Selecting the exact value is a trade-off with engeneering concerns. In some degrees of freedom, the alignment may be easily reached by machining tolerances and thus we can chose a number that causes only a negligible degradation of performance, while in other cases, reaching a certain alignment might be very costly and thus we want to set the requirements for this parameters as loosely as possible.

In many cases, the effects of different misalignments will just linearly add up, in others they may cancel each other out to some degree or combine multiplicatively. In a second step, we investigate misalignments for all parameters simulataneously, but instead of exhaustivly covering the entire parameter space, we perform simualtions only for the table of aligment requirements developed in Step 1. Assuming that misaligments are normally distributed, we draw many possible realizations of the PiSoX instrument from our alignment table. For each realization, we run a ray-trace and calculate the performance. We look at the distribution of results and decide if these fullfill the science requirements or if certain alignment values have to be tightened.

For the purpose of developing the error budget, there are other design parameters that are not technically related to mechnical aligment, but impact the performce in a similar way and can be analyzed with the same ray-trace setup. One example is the pointing jitter, which describes how uncertainties in the instrument pointing on the sky degrade the instrument performance. If the pointing direction on the sky jitters with time, photons will not always arrive on-axis. This is somewhat similar to a misaligned optics module.

Ray-traces are performed with the MARXS code.
Every ray-trace has limitations. The most important one for this work is that in the current model, the PiSoX mirror is approximated by a flat, perfect lens with additional scatter, instead of a true, 3D represenation of the mirror surface. While the scatter is tuned to give a PSF of the correct size in the focal plane, the shape of the PSF differs from what a single parabolic mirror will give, particularly for off-axis source.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import astropy.units as u
from marxs.design.tolerancing import select_1dof_changed

%matplotlib inline

In [ ]:
datpath = '../run_results/pisox/'

In [ ]:
import sys
sys.path.append('..')
from redsox.tolerances import plot_wiggle, load_and_plot

## Step 1

In this section, we run simulations varying one degree of freedom at a time and present results in three different kinds of plots: First, there are parameters that are not mechanical misalignments, such as the pointing jitter. Figures showing the results of these simulations have a blue background. Simulations for mechanical misalignments come in two flavors. Either an entire set of objects is moved determinsitically (e.g. all gratings in the grating assembly are moved 1 mm to the right) or a number of objects are moved randomly (e.g. all gratings in the grating assembly are moved along the $x$-axis, but for each grating a new number is drawn from a Gaussian distribution with $\sigma=1$ mm). The first case is shown with a gray background, the latter case is shown with a pink background. Results for all mechnical tolerancing are shown as sets of six plots. The upper row presents results from translations along the $x$, $y$, and $z$-axis, the bottom row rotations. The center of the rotation is typically the centor of an element. If other centers are chosen, this is discussed in the text. The coordinate system for PiSoX places the optical axis along the $z$-axis with photons coming in from $z=+\infty$. The origin of the coordinate system is at the nominal focal point of the mirror system. The dispersion direction of the gratings is along the positive $y$-axis of PiSoX. Thus, the long axis of the ML mirror is also parallel to the $y$-axis.

Each plot shows different lines. Solid lines show the change of the modulation factor with changes of a parameter and correspond to the numbers on the left $y$-axis of a plot, while changes in the effective area are shown with dotted lines corresponding to the right $y$-axis of the plot. Note that the scale on each plot is chosen to highlight the relevant parameter space and can differ from plot to plot. In many cases, there is relatively little change in the predicted modulation factor and thus the range of values shown along the left $y$-axis is very small. In these cases, the line just scatters around the average value due to the Poisson error in the simulations. 

### Pointing and mirror

In [ ]:
t = Table.read(os.path.join(datpath, 'jitter.fits'))
fig = plt.figure()
ax = fig.add_subplot(111)

#t = select_1dof_changed(tab, par, ['jitter'])
t_wave = t.group_by('wave')
axt = ax.twinx()
    
for key, g in zip(t_wave.groups.keys, t_wave.groups):
    x = g['jitter']
    ax.set_title('Pointing jitter')
    ax.plot(x, np.abs(g['modulation'][:, 1]), label='{:3.1f} $\AA$'.format(key[0]), lw=1.5)
    axt.plot(x, g['aeff'][:, 1], ':', label='{:2.0f} $\AA$'.format(key[0]), lw=2)
    ax.set_ylabel('Modulation factor (solid lines)')
    axt.set_ylabel('$A_{eff}$ [cm$^2$] per channel (dotted lines)')
    ax.set_xlabel('jitter $\sigma$ in arcmin')
ax.set_facecolor((0.9, 0.9, 1.))
ax.set_axisbelow(True)
ax.grid(axis='x', c='1.0', lw=2, ls='solid')
out = ax.legend()

This figure shows simulations using an unsteady pointing. The average pointing direction is on-axis, but the pointing jitters around that. For each photon, the true pointing direction is drawn from a Gaussian with the $\sigma$ given in the figure. This jitter represents uncertainty in the pointing, which can come from different sources, such as limited resolution of the star trackers, motion of the pointing within the time period of reading out the star trackers or integetration time of the zero-order image (if used to determine the target coordinates) or the spacecraft not correcting a pointing drift fast enough.

The effective area $A_{\mathrm{eff}}$ drops with increasing jitter, because the diffracted photons do not hit the ML at the position of the Bragg peak when the target is not at a nominal position, and thus the reflectivity is lower. The drop becomes important for a jitter above a few arcminutes. Mispointing along the direction of diffraction has a much stronger effect than perpendicular to it. This is investigated in the next figure.

In [ ]:
tab = Table.read(os.path.join(datpath, 'offset_point.fits'))
# "Default" pointing is at (30, 0)
tab['ra_offset'] = (tab['coords'].ra - 30 * u.degree).to(u.arcmin).value
tab['dec_offset'] = tab['coords'].dec.to(u.arcmin).value
tab.remove_column('coords')
fig = plt.figure(figsize=(10, 5))

for i, par in enumerate(['ra_offset', 'dec_offset']):
    t = select_1dof_changed(tab, par, ['ra_offset', 'dec_offset'])
    ax = fig.add_subplot(1, 2, i+1)
    t_wave = t.group_by('wave')
    axt = ax.twinx()
    
    for key, g in zip(t_wave.groups.keys, t_wave.groups):
        g.sort(par)
        x = g[par]
        ax.set_title({'ra_offset': 'Offset along short axis of ML',
                     'dec_offset': 'Offset along long axis of ML'}[par])
        ax.plot(x, np.abs(g['modulation'][:, 1]), label='{:3.1f} $\AA$'.format(key[0]), lw=1.5)
        axt.plot(x, g['aeff'][:, 1], ':', label='{:2.0f} $\AA$'.format(key[0]), lw=2)
        ax.set_ylabel('Modulation factor (solid lines)')
        axt.set_ylabel('$A_{eff}$ [cm$^2$] per channel (dotted lines)')
        ax.set_xlabel('offset of point source [arcmin]')
    ax.set_facecolor((0.9, 0.9, 1.))
    ax.set_axisbelow(True)
    ax.grid(axis='x', c='1.0', lw=2, ls='solid')
    ax.legend()
    #out = ax.set_xlim([np.min(x), None])

out = ax.set_xlim([-2, 2])
fig.subplots_adjust(wspace=.5)

The modulation factor changes only marginally when the source is observed offset from the nominal position. However, as explained for the simulations with the pointing jitter above, the effective area drops dramatically when the source  moves along the axis of the ML, because that means that photons will no longer arrive at the position where the spacing of the ML matches the required number given the angle and wavelength of the photon. Because photons with a longer wavelength are more dispersed, they are more effected.

In [ ]:
tab = Table.read(os.path.join(datpath, 'scatter.fits'))
fig = plt.figure(figsize=(12, 5))

for i, par in enumerate(['inplanescatter', 'perpplanescatter']):
    t = select_1dof_changed(tab, par, ['inplanescatter', 'perpplanescatter'])
    ax = fig.add_subplot(1, 2, i+1)
    t_wave = t.group_by('wave')
    axt = ax.twinx()
    
    for key, g in zip(t_wave.groups.keys, t_wave.groups):
        x = np.rad2deg(g[par])*60
        ax.set_title(par)
        ax.plot(x, np.abs(g['modulation'][:, 1]), label='{:3.1f} $\AA$'.format(key[0]), lw=1.5)
        axt.plot(x, g['aeff'][:, 1], ':', label='{:2.0f} $\AA$'.format(key[0]), lw=2)
        ax.set_ylabel('Modulation factor (solid lines)')
        axt.set_ylabel('$A_{eff}$ [cm$^2$] per channel (dotted lines)')
        ax.set_xlabel('Gaussian $\sigma$ in arcmin')
    ax.set_facecolor((0.9, 0.9, 1.))
    ax.set_axisbelow(True)
    ax.grid(axis='x', c='1.0', lw=2, ls='solid')
    ax.legend()
    out = ax.set_xlim([np.min(x), None])
fig.subplots_adjust(wspace=.5)

This figure shows how the mirror scatter changes the performance of PiSoX. However, given the extreme simplicity of the currently implemented mirror model, this result should be interpreted with caution.

### CAT gratings

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'CAT_global.fits'))
fig.subplots_adjust(wspace=.7)

This figure shows simulations that move the CAT grating modules as a whole, i.e. translation in $z$ means that all gratings of both sectors are moved up or down together. This particular case changes the distance between the gratings and the focal plane and thus photons will hit the ML mirror on a different location. Changes of more than a few mm will cause the photons to miss the position of the Bragg peak on the ML mirror and thus reduce $A_{\mathrm{eff}}$. The layout is insentitive to translations in $y$ (along the dispersion direction). This is the long direction of the CAT gratings and CAT gratings are tilted only ever so slightly, so that the point of intersection is essentially constant. $A_{\mathrm{eff}}$ only begins to drop, when gratings are moved so far that some fraction of the beam no longer hit a grating. A shift along $x$ is a shift along the stair-stepped direction. Shifts along $x$ reduce $A_{\mathrm{eff}}$ for the same reason that the layout is stair-stepped in the first place: Since photons come in at a different angle, they need to be diffracted at a different distance from the ML to hit the ML mirror at the Bragg peak.

For the rotation simulations, the origin of the rotation is the point where the optical axes intersects the "stair" surface on which the gratings are positioned. Of all the rotations, only rotations around the $y$ direction (the dispersion axis) have limits tighter than a degree or so, because rotation around $y$ changes the $z$ position of the gratings, so the effect is similar to a translation in $z$.

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'CAT_individual.fits'))
fig.subplots_adjust(wspace=.7)

The figure above presents simulations where individual CAT gratings are moved with respect to their nominal position on the CAT gratings assembly, i.e. where the grating holders are not positioned correctly. All translations allow $1\sigma$ errors of a few mm, which is much larger than the size of the holder the gratings are placed in. This is a trivial constraint. Similarly, only rotations around $x$ (the short axis of the gratings) is tighter than 2 degrees. Rotations around $x$ makes the incoming photons hit the CAT gratings at an angle different from the design blaze angle and reduce the fraction of photons that are dispersed into the first order. Since photons in other orders are not reflected from the ML mirror onto the detector, this reduces the $A_{\mathrm{eff}}$ of the system. Longer wavelengths drop faster. To keep the loss of $A_{\mathrm{eff}}$ below 10%, the gratings need to be positioned within 10 arcmin of the nominal rotation angle.

In [ ]:
t = Table.read(os.path.join(datpath, 'CAT_period.fits'))
fig = plt.figure()
ax = fig.add_subplot(111)
t_wave = t.group_by('wave')
axt = ax.twinx()
    
for key, g in zip(t_wave.groups.keys, t_wave.groups):
    x = g['period_sigma'] / g['period_mean']
    ax.set_xlabel('relative change in grating period')
    
    ax.plot(x, np.abs(g['modulation'][:, 1]), label='{:3.1f} $\AA$'.format(key[0]), lw=1.5)
    axt.plot(x, g['aeff'][:, 1], ':', label='{:2.0f} $\AA$'.format(key[0]), lw=2)
    ax.set_ylabel('Modulation factor (solid lines)')
    axt.set_ylabel('$A_{eff}$ [cm$^2$] per channel (dotted lines)')


ax.set_facecolor((0.9, 0.9, 1.))
ax.set_axisbelow(True)
ax.grid(axis='x', c='1.0', lw=2, ls='solid')
ax.legend()
ax.set_xscale("log")
ax.set_title('Variation of the grating period')
out = ax.set_xlim([np.min(x), None])

A change in the period of the gratings will also diffract photons to the wrong locations, but the lithography process used to manufacture the gratings gives a repeatability of the grating period that is orders of magnitude better than the PiSoX requirement.

### Multi-layer mirror

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'LGML_global.fits'))
axes = fig.axes
axes[2].set_xlim(-.1, .1)
axes[4].set_xlim(-2, 2)
axes[8].set_xlim(-.1, .1)
axes[10].set_xlim(-1, 1)

As seen above, the ML is the most critical part of the alignment because photons have to hit the multilayer at the position of the Bragg peak. Because of this, shifts along the $y$ direction (the direction in which the multilayer is graded) are most sensitive and have to be aligned to better than 10 micron. However, this is for a source at nominal position. As see above, moving the source to a slightly off-axis position also changes where photons interact with the ML mirror. Thus, in practice, this alignment does not have to be performed to the 10 micron level. Instead sources can just be observed slightly off-axis to compensate for any alignment error. This requires calibrating the alignment in space by observing a source at different positions, until the signal in the polarimetry channel is maximised. In this case, the instrument can no longer rotate around the nominal axis to probe different polarization angles. Instead, it has to rotate such that the source is kept at the new position determined in the calibration. On the other hand, tolerances for the other translations are a lot more relaxed around a mm or so.

Rotations around the long axis of the ML mirror ($x$ axis of the coordinate system) have a very large tolerance of a few degrees. Because the physical dimensions of the mirror are small, the point of intersection with the mirror surface does not change much and thus the photons still interact with the mirror very close to position of the Bragg peak. On the other hand, rotations around the other two axes move the mirror by a significant amount. That means that the photons travel either too far or too little and since the photons are dispersed, they will also travel to far or not far enough in $x$ direction, which causes them to miss the position of the Bragg peak and consequently reduces $A_{\mathrm{eff}}$ significantly.


**Move origin of the rotation to center of ML mirror.**

In [ ]:
t = Table.read(os.path.join(datpath, 'LGML_gradient.fits'))
fig = plt.figure()
ax = fig.add_subplot(111)
t_wave = t.group_by('wave')
axt = ax.twinx()
    
for key, g in zip(t_wave.groups.keys, t_wave.groups):
    x = g['lateral_gradient']
    ax.set_xlabel('lateral gradient of LGML')
    
    ax.plot(x, np.abs(g['modulation'][:, 1]), label='{:3.1f} $\AA$'.format(key[0]), lw=1.5)
    axt.plot(x, g['aeff'][:, 1], ':', label='{:2.0f} $\AA$'.format(key[0]), lw=2)
    ax.set_ylabel('Modulation factor (solid lines)')
    axt.set_ylabel('$A_{eff}$ [cm$^2$] per channel (dotted lines)')


ax.set_facecolor((0.9, 0.9, 1.))
ax.set_axisbelow(True)
ax.grid(axis='x', c='1.0', lw=2, ls='solid')
ax.legend()
ax.set_xscale("log")
ax.set_title('Gradient of LGML')
out = ax.set_xlim([np.min(x), None])

### Detectors

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'detector_global.fits'))
fig.subplots_adjust(wspace=.7)

The exact position of the detectors is not important for PiSoX as long as the signal still hits the detectors. Two effects of detector misalignment are not shown here: With increasing misalignment, the spectral resolution of the polarimetry channel will degrade slightly (not shown here), but the low signal in this channel will prevent an analysis of high-resolution spectroscopy anyway. Also, background also increases, when the extraction region size needs to be increased, but again, this effect is nigligible for any misalignment that can reasonably be expected in the focal plane.

## Repeat plots with figure of merrit (FOM)

The figure of merrit for a spectro-polarimeter combines effective area and modulation into a single number. In this section, some plots are repeated to show the FOM instead of individual lines for modulation factor and effective area.

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'CAT_global.fits'), plot_type='MDP')
fig.axes[3].set_xlim([-1, 1])
out = fig.axes[4].set_xlim([-1, 1])
fig.subplots_adjust(wspace=.3)

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'CAT_individual.fits'), plot_type='MDP')
fig.subplots_adjust(wspace=.3)

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'LGML_global.fits'), plot_type='MDP')
fig.axes[1].set_xlim([-.1, .1])
fig.axes[4].set_xlim([-.1, .1])
out = fig.axes[5].set_xlim([-1, 1])
fig.subplots_adjust(wspace=.3)

## Step 2: Total alignment budget

To be done at a later phase.

## Plots with detailed formatting
This section repeats a few plots from above with specfic formatting. For proposals or publications, we often need to tweak the size of labels, fonts or other formatting by hand. Some of those plots are below, based on the same data as above.

In [ ]:
tab = Table.read(os.path.join(datpath, 'offset_point.fits'))
# "Default" pointing is at (30, 0)
tab['ra_offset'] = (tab['coords'].ra - 30 * u.degree).to(u.arcmin)
tab['dec_offset'] = tab['coords'].dec.to(u.arcmin)
tab.remove_column('coords')

fig, axes = plt.subplots(1, 2, sharey=True)
# Remove horizontal space between axes
fig.subplots_adjust(wspace=0)

for i, par in enumerate(['ra_offset', 'dec_offset']):
    t = select_1dof_changed(tab, par, ['ra_offset', 'dec_offset'])
    t_wave = t.group_by('wave')
    
    for key, g in zip(t_wave.groups.keys, t_wave.groups):
        g.sort(par)
        x = g[par]
        axes[i].set_title({'ra_offset': 'cross-dispersion',
                     'dec_offset': 'dispersion'}[par])
        axes[i].plot(x, g['aeff'][:, 1],  label='{:2.0f} $\AA$'.format(key[0]), lw=2)
        axes[i].set_xlabel('pointing offset [arcmin]')
        axes[i].grid()
axes[0].legend()
axes[0].set_ylabel('$A_{eff}$ in polarimetry channel [cm$^2$]')
axes[1].set_xlim(-1.5, 1.5)
    #out = ax.set_xlim([np.min(x), None])
axes[0].set_facecolor((0.9, 0.9, 1.))
axes[1].set_facecolor((0.9, 0.9, 1.))
    
#fig.savefig('../pisoxplots/offaxis.pdf', bbox_inches='tight')
#fig.savefig('../pisoxplots/offaxis.png', bbox_inches='tight')
fig.savefig('/Users/hamogu/MITDropbox/my_poster/20_SPIE_Polarimetry/SPIE2020-GoPiSox/offset_point.pdf', bbox_inches='tight')
fig.savefig('/Users/hamogu/MITDropbox/my_poster/20_SPIE_Polarimetry/offset_point.png', bbox_inches='tight', dpi=300)

In [ ]:
t = Table.read(os.path.join(datpath, 'jitter.fits'))
fig = plt.figure()
ax = fig.add_subplot(111)

#t = select_1dof_changed(tab, par, ['jitter'])
t_wave = t.group_by('wave')
    
for key, g in zip(t_wave.groups.keys, t_wave.groups):
    x = g['jitter']
    ax.set_title('Pointing jitter')
    ax.plot(x, g['aeff'][:, 1], label='{:2.0f} $\AA$'.format(key[0]), lw=2)
    ax.set_ylabel('$A_{eff}$ [cm$^2$] per channel (dotted lines)')
    ax.set_xlabel('jitter $\sigma$ in arcmin')
ax.set_facecolor((0.9, 0.9, 1.))
ax.set_axisbelow(True)
ax.grid(axis='x', c='1.0', lw=2, ls='solid')
out = ax.legend()
fig.savefig('/Users/hamogu/MITDropbox/my_poster/20_SPIE_Polarimetry/SPIE2020-GoPiSox/jitter.pdf', bbox_inches='tight')

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'CAT_global.fits'), plot_type='aeff')
fig.subplots_adjust(wspace=.7)
fig.savefig('/Users/hamogu/MITDropbox/my_poster/20_SPIE_Polarimetry/SPIE2020-GoPiSox/CAT_global.pdf', bbox_inches='tight')

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'CAT_individual.fits'), plot_type='aeff')
fig.subplots_adjust(wspace=.7)
fig.savefig('/Users/hamogu/MITDropbox/my_poster/20_SPIE_Polarimetry/SPIE2020-GoPiSox/CAT_individual.pdf', bbox_inches='tight')

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'LGML_global.fits'), plot_type='aeff')
axes = fig.axes
axes[1].set_xlim(-.1, .1)
axes[2].set_xlim(-2, 2)
axes[4].set_xlim(-.1, .1)
axes[5].set_xlim(-1, 1)
fig.savefig('/Users/hamogu/MITDropbox/my_poster/20_SPIE_Polarimetry/SPIE2020-GoPiSox/LGML_global.pdf', bbox_inches='tight')

In [ ]:
t = Table.read(os.path.join(datpath, 'LGML_gradient.fits'))
fig = plt.figure()
ax = fig.add_subplot(111)
t_wave = t.group_by('wave')
    
for key, g in zip(t_wave.groups.keys, t_wave.groups):
    x = g['lateral_gradient'] / 1.6e-7
    ax.set_xlabel('lateral gradient of ML mirror relative to design value')
    ax.plot(x, g['aeff'][:, 1], label='{:2.0f} $\AA$'.format(key[0]), lw=2)
    ax.set_ylabel('$A_{eff}$ [cm$^2$]')


ax.set_facecolor((0.9, 0.9, 1.))
ax.set_axisbelow(True)
ax.grid(axis='x', c='1.0', lw=2, ls='solid')
ax.legend()
#ax.set_xscale("log")
ax.set_title('Gradient of ML mirror')
out = ax.set_xlim([np.min(x), None])
fig.savefig('/Users/hamogu/MITDropbox/my_poster/20_SPIE_Polarimetry/SPIE2020-GoPiSox/LGML_gradient.pdf', bbox_inches='tight')